# 1. Import Dependencies

In [1]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

# 2. Test Environment

[Download Atari Rom](www.atarimania.com/rom/Roms.rar)

[Install atari-py](https://github.com/openai/atari-py)

Pip install didn't work for me, [conda installed](https://anaconda.org/conda-forge/atari_py) with `conda install -c conda-forge atari_py` 

In [2]:
!python -m atari_py.import_roms ./ROMS

copying asterix.bin from ./ROMS/Asterix (AKA Taz) (1984) (Atari, Jerome Domurat, Steve Woita) (CX2696).bin to /home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/atari_py/atari_roms/asterix.bin
copying mr_do.bin from ./ROMS/Mr. Do! (1983) (CBS Electronics, Ed English) (4L4478) (PAL).bin to /home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/atari_py/atari_roms/mr_do.bin
copying star_gunner.bin from ./ROMS/Stargunner (1983) (Telesys, Alex Leavens) (1005) ~.bin to /home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/atari_py/atari_roms/star_gunner.bin
copying video_pinball.bin from ./ROMS/Pinball (AKA Video Pinball) (Zellers).bin to /home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/atari_py/atari_roms/video_pinball.bin
copying bowling.bin from ./ROMS/Bowling (1979) (Atari, Larry Kaplan - Sears) (CX2628 - 6-99842, 49-75117) ~.bin to /home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/atari_py/atari_roms/bowling.bin
copying road_runner.bin from patched v

In [3]:
environment_name = 'Breakout-v0'
env = gym.make(environment_name)

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [4]:
env.reset()

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [5]:
print(type(env.action_space))
print(env.action_space.n)

<class 'gym.spaces.discrete.Discrete'>
4


In [6]:
print(type(env.observation_space))
print(env.observation_space.shape)

<class 'gym.spaces.box.Box'>
(210, 160, 3)


In [7]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print(f'Episode {episode} score: {score}')
env.close()

/home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/gym/envs/atari/environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1 score: 2.0
Episode 2 score: 4.0
Episode 3 score: 1.0
Episode 4 score: 2.0
Episode 5 score: 1.0


# 3. Vectorise Environment and Train Model

In [8]:
env = make_atari_env('Breakout-v0', n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

In [9]:
log_path = os.path.join('Training', 'Logs')
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [17]:
timesteps = 1e6
model.learn(total_timesteps=timesteps)

Logging to Training/Logs/A2C_7
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 515      |
|    ep_rew_mean        | 6.44     |
| time/                 |          |
|    fps                | 363      |
|    iterations         | 100      |
|    time_elapsed       | 5        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -0.521   |
|    explained_variance | 0.875    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5099     |
|    policy_loss        | 0.0262   |
|    value_loss         | 0.0395   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 512      |
|    ep_rew_mean        | 6.32     |
| time/                 |          |
|    fps                | 366      |
|    iterations         | 200      |
|    time_elapsed       | 10       |
|    total_timesteps    | 4000     |
| train

# 4. Save and Reload Model

In [18]:
a2c_path = os.path.join('Training', 'Saved_Models', f'A2C_Breakout_model_{timesteps}_timesteps')
model.save(a2c_path)

In [19]:
del model

In [20]:
model = A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


# 5. Evaluate and Test

In [21]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [23]:
avg, std = evaluate_policy(model, env, n_eval_episodes=10, render=True)
print(f'Average: {avg}\nSTD: {std}')
env.close()

Average: 12.8
STD: 3.7094473981982814
